# **O uso de aprendizado de máquina para a recomendação de filmes**

Grupo 05 - Lucas Bastelli Spagnol, Pedro Gabriel Gonçalves

# Importando as bibliotecas 

In [ ]:
# Para manipulação e visualização
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('ggplot')

# Para pré-processamento dos textos
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
import re

# para extração de features
from sklearn.feature_extraction.text import TfidfVectorizer

# para similaridades
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
import json

from tqdm.notebook import tqdm
from time import sleep

# Importando e conhecendo o conjunto de dados

Vamos importar dois conjuntos de dados disponíveis, o primeiro conjunto de dados (tmdb_5000_credits.csv) contém as seguintes informações:

* movie_id: Um único identificador para cada filme.

* elenco: O nome dos atores principais e coadjuvantes.

* equipe: O nome do Diretor, Editor, Compositor, Escritor etc.


O segundo conjunto de dados (tmdb_5000_movies.csv) contém as seguintes informações:


* orçamento: O orçamento em que o filme foi feito.

* gênero: O gênero do filme, ação, comédia, suspense etc.

* homepage: Um link para a página inicial do filme.

* id: Este é, de fato, o movie_id como no primeiro conjunto de dados.

* palavras-chave: As palavras-chave ou tags relacionadas ao filme.

* original_language: O idioma em que o filme foi feito.

* original_title: O título do filme antes da tradução ou adaptação.

* visão geral: Uma breve descrição do filme.

* popularidade: Uma quantidade numérica que especifica a popularidade do filme.

* production_companies: A casa de produção do filme.

* production_countries: O país em que foi produzido.

* release_date: Um dado que foi lançado.

* receita: A receita gerada pelo filme.

* runtime: O tempo de execução do filme em minutos.

* status: "Lançado" ou "Rumor".

* slogan: Slogan do filme.

* título: Título do filme.

* vote_average: classificações médias que o filme recebido.

* vote_count: uma contagem dos votos recebidos.

In [ ]:
# importando o conjunto de dados
df_credit = pd.read_csv('/content/tmdb_5000_credits.csv')
df_movies = pd.read_csv('/content/tmdb_5000_movies.csv')

# convertendo a data em modo datetime
df_movies['release_date'] = pd.to_datetime(df_movies.release_date)

# verificando as primeiras linhas
df_credit.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [ ]:
# verificando as primeiras linhas
df_movies.head(2)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [ ]:
# checando as dimensões
print(df_movies.shape)
print(df_credit.shape)

(4803, 20)
(4803, 4)


In [ ]:
# checando se há dados faltantes
df_credit.isnull().sum()

movie_id    0
title       0
cast        0
crew        0
dtype: int64

In [ ]:
# checando se há dados faltantes
pd.DataFrame({'faltas_abs':df_movies.isnull().sum(),
              'prop_falta':df_movies.isnull().sum()/df_movies.shape[0]})

,faltas_abs,prop_falta
budget,0,0.000000
genres,0,0.000000
homepage,3091,0.643556
id,0,0.000000
keywords,0,0.000000
original_language,0,0.000000
original_title,0,0.000000
overview,3,0.000625
popularity,0,0.000000
production_companies,0,0.000000


O primeiro conjunto de dados não constam dados faltantes, já no segundo sim. Verificamos que a coluna homepage tem aproximadamente 65%, nesse caso não será problema porque não vamos utilizar essa coluna, então poderemos descarta-la.

Agora vamos olhar se os dados foram importados com seus respectivos tipos corretos.

In [ ]:
# checando os tipos dos dados
df_credit.dtypes

movie_id     int64
title       object
cast        object
crew        object
dtype: object

In [ ]:
# checando os tipos dos dados
df_movies.dtypes

budget                           int64
genres                          object
homepage                        object
id                               int64
keywords                        object
original_language               object
original_title                  object
overview                        object
popularity                     float64
production_companies            object
production_countries            object
release_date            datetime64[ns]
revenue                          int64
runtime                        float64
spoken_languages                object
status                          object
tagline                         object
title                           object
vote_average                   float64
vote_count                       int64
dtype: object

# Extraindo features (recursos)

Vamos fazer algumas extrações para facilitar as análises, à exemplo vimos que a coluna genres contém um dicionário em cada célula e isso não nos dá uma forma otimizada de análise.

In [ ]:
# removendo a coluna homepage
df_movies.drop(['homepage'], axis=1, inplace=True)

Vamos criar uma coluna de descrição, a coluna tagline possui algumas células em branco, então vamos preencher os dados null com vazios, e juntar com a coluna overview. Vamos fazer isso porque das células em branco, juntando com outra não vamos perder informação quando remover.

In [ ]:
# preenchendo os vazios
df_movies.tagline.fillna('', inplace=True)

# criando uma coluna de descrição
df_movies['description'] = df_movies['tagline'].map(str) + ' ' + df_movies['overview']

# removendo valores nulos
df_movies = df_movies.dropna().reset_index().drop('index', axis=1)

# checando os dados faltantes
df_movies.isnull().sum()

budget                  0
genres                  0
id                      0
keywords                0
original_language       0
original_title          0
overview                0
popularity              0
production_companies    0
production_countries    0
release_date            0
revenue                 0
runtime                 0
spoken_languages        0
status                  0
tagline                 0
title                   0
vote_average            0
vote_count              0
description             0
dtype: int64

Não temos mais dados faltantes, vamos checar pra ver quantas linhas perdemos nessa transformação.

In [ ]:
# checando o dataframe com as novas dimensões
df_movies.shape

(4799, 20)

Anteriormente tínhamos 4803 linhas e agora temos 4799, não perdemos quase nada.

Vamos querer também analisar por ano e mês de lançamento, então vamos criar mais essas duas colunas.

In [ ]:
#Acho q dá para tirar isso (caso tire tem q tirar o comentário acima (Vamos querer...))
df_movies['release_year'] = df_movies.release_date.dt.year
df_movies['release_month'] = df_movies.release_date.dt.month

Para extrairmos as colunas que possuem dicionários, vamos criar duas funções utilizando funções json que facilitam as extrações das informações nesse formato.

In [ ]:
# função para extrair os nomes em cada linha de determinada coluna
def extract_names(coluna):
    
    # colocando o coluna em uma variável
    col_name = f"{coluna}"
    
    # criando um dataframe vazio
    df_coluna=pd.DataFrame()
    
    # colocando um valor para iniciar a contagem no loop
    index=1
    
    # iterando com um loop na coluna passada como argumento
    for i,each in enumerate(df_movies[f'{coluna}']):
        
        
        coluna_list=json.loads(each)
        for coluna in coluna_list:
            df_coluna=pd.concat([df_coluna,pd.DataFrame({"coluna":coluna["name"],
                                                         "movie_id": df_movies.id[i],
                                                         "original_title":df_movies.original_title[i],
                                                         "popularity":df_movies.popularity[i],
                                                         "revenue":df_movies.revenue[i],
                                                         "budget":df_movies.budget[i],
                                                         "release_date":df_movies.release_date[i]},index=[index])],axis=0)
            
            index+=1
    
    df_coluna.rename(columns={'coluna':col_name}, inplace=True)
    
    return df_coluna

#=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=#

# função para extrair os nomes dos dicionarios em cada linha de determinada coluna
def json_decode(data,key):
    result = []
    data = json.loads(data) #convert to jsonjsonn from string
    for item in data: #convert to list from json
        result.append(item[key])
    return result

Agora vamos criar alguns dataframes somente com as informações pertinentes que queremos analisar, mas se quisermos agregar com informações de outros dataframes criados, podemos juntar através da coluna ID.

In [ ]:
# aplicandao a primeira função e extraindo as informações principais de cada dicionário
df_credit['character'] = df_credit.cast.apply(json_decode, key='character')
df_credit['name_character'] = df_credit.cast.apply(json_decode, key='name')
df_credit['department_crew'] = df_credit.crew.apply(json_decode, key='department')
df_credit['job_crew'] = df_credit.crew.apply(json_decode, key='job')
df_credit['name_crew'] = df_credit.crew.apply(json_decode, key='name')

In [ ]:
# checando as duas primeiras linhas
df_credit.head(2)

,movie_id,title,cast,crew,character,name_character,department_crew,job_crew,name_crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[Jake Sully, Neytiri, Dr. Grace Augustine, Col...","[Sam Worthington, Zoe Saldana, Sigourney Weave...","[Editing, Art, Sound, Sound, Production, Sound...","[Editor, Production Design, Sound Designer, Su...","[Stephen E. Rivkin, Rick Carter, Christopher B..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","[Captain Jack Sparrow, Will Turner, Elizabeth ...","[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[Camera, Directing, Production, Writing, Writi...","[Director of Photography, Director, Producer, ...","[Dariusz Wolski, Gore Verbinski, Jerry Bruckhe..."


In [ ]:
df_movies.head(2)

,budget,genres,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,description,release_year,release_month
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Enter the World of Pandora. In the 22nd centur...,2009,12
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,"At the end of the world, the adventure begins....",2007,5


In [ ]:
df_movies_full = pd.merge(df_movies, df_credit[['movie_id', 'character', 'name_character', 'department_crew']], left_on='id', right_on='movie_id').drop('movie_id', axis=1)

Agora iremos começar a a criar o nosso modelo.

Esse modelo é baseado em conteúdo, utilizando técnicas de NLP e aplicando a função coseno para retornar os filmes mais similares, abaixo descrevo brevemente sobre essa função.

Vamos iniciar selecionando as colunas principais para o nosso modelo.

In [ ]:
# mantendo as principais colunas
df1 = df_movies_full[['title', 'tagline', 'overview', 'popularity', 'description']]

# Construindo o Sistema Recomendador de filmes

Os seguintes passos serão seguidos:

- Pré-processamento do texto

- Engenharia de features

- Computar as similaridades dos documentos

- Encontrar os tops filmes similares

- Construir a função para recomendação de filmes

# Pré-processamento do texto

É a tarefa de deixar o texto analisável, a cada análise que envolva NLP (Processamento de Linguagem Natural) é necessário avaliar as técnicas que serão empregadas, pois não há uma única forma para utilizar em tudo. Por exemplo, para analisar as palavras mais comuns que aparecem em determinado documento, se remover stopwords pode ser que eu remova outras informações importantes.

Também conhecido como normalização de textos, na célula abaixo trataremos de criar uma função para isso.

Como o texto está em inglês, removeremos stopwords desse idioma, assim como caracteres especiais e passaremos todas as palavras para minúsculas e também removeremos espaços que estão a mais.

In [ ]:

nltk.download('stopwords')
# objeto para stopwords
stop_words = stopwords.words('english')

# função para normalização do texto
def normalize_doc(doc):
    # removendo caracteres especiais
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    
    # convertendo em minusculas
    doc = doc.lower()
    
    # removendo espaços indesejados no final
    doc = doc.strip()
    
    # tokenizando os documentos
    tokens = word_tokenize(doc)
    
    # filtrando as stopwords
    filtered_tokens = [token for token in tokens if token not in stop_words]
    
    # juntando o tokens com o texto limpo
    doc = ' '.join(filtered_tokens)
    
    # resultado final
    return doc

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Antes de aplicar a função vamos criar uma instância vetorizando a função para aplicar nos documentos, com o método vectorize do numpy, mas o que essa função faz? Ela pega objetos ou matrizes como entrada e retorna uma única matriz numpy, no nosso caso será retornado uma matriz com os textos normalizados.

In [ ]:
# instanciando um objeto para vetorizar a função criada
normalize_corpus = np.vectorize(normalize_doc)

Agora vamos executar a função e ver dar uma olhada em sua dimensão.

In [ ]:
nltk.download('punkt')
# executando a função
norm_corpus = normalize_corpus(list(df1['description']))

#  olhando o tamanho do corpus
norm_corpus.shape

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


(4799,)

Eis o resultado da vetorização, ele converteu um objeto de uma coluna do dataframe em um array numpy com os documentos.

# TF-IDF

In [ ]:
# instanciando o tf-idf
tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)

# treinando e transformando o corpus
tfidf_matrix = tf.fit_transform(norm_corpus)

# checando as dimensões da matriz
tfidf_matrix.shape

(4799, 20659)

A matriz sparsa resultante possui 20659 colunas e a quantidade de linhas se mantém.

# Computando as similaridades

In [ ]:
# calculando as similaridades na matriz
doc_sim = cosine_similarity(tfidf_matrix) #Cosseno
doc_eucl = euclidean_distances(tfidf_matrix) #Euclidiana
# colocando em um dataframe
doc_sim_df = pd.DataFrame(doc_sim) 
doc_eucl_df = pd.DataFrame(doc_eucl) 

# verificando as primeiras linhas
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,...,4789,4790,4791,4792,4793,4794,4795,4796,4797,4798
0,1.000000,0.010701,0.00000,0.019029,0.028685,0.0249,0.000000,0.026516,0.000000,0.007419,...,0.009701,0.0,0.023336,0.033549,0.000000,0.000000,0.0,0.006889,0.000000,0.000000
1,0.010701,1.000000,0.01189,0.000000,0.041622,0.0000,0.014562,0.027121,0.034687,0.007614,...,0.009955,0.0,0.004817,0.000000,0.000000,0.012592,0.0,0.022382,0.013724,0.000000
2,0.000000,0.011890,1.00000,0.000000,0.000000,0.0000,0.000000,0.022242,0.015853,0.004891,...,0.042615,0.0,0.000000,0.000000,0.016519,0.000000,0.0,0.011677,0.000000,0.003999
3,0.019029,0.000000,0.00000,1.000000,0.008792,0.0000,0.015973,0.023171,0.027451,0.073607,...,0.000000,0.0,0.009666,0.000000,0.000000,0.000000,0.0,0.028343,0.021783,0.027732
4,0.028685,0.041622,0.00000,0.008792,1.000000,0.0000,0.022909,0.028676,0.000000,0.023536,...,0.014799,0.0,0.000000,0.000000,0.000000,0.010760,0.0,0.010509,0.000000,0.000000


Foram calculados a similaridades em cada linha contra cada linha do dataframe, semelhante a uma matriz de correlação, as linhas 0 e 0 por exemplo tem score 1 porque é ela própria e a similaridade tem o valor máximo.

In [ ]:
# convertendo a coluna title em uma lista
movies_list = df1.title.values

# olhando a dimensão
movies_list.shape

(4799,)

#Encontrando os mais similares com base em uma amostra

-------------------------------------------
Acho que poderiamos tirar essa parte e ir direito pro recomendador de filme
Se vc concordar em tirar, além de tirar aqui, deve tirar essa parte na Descrição e no Slide

In [ ]:
# ordenando por popularidade (do mais popular)
pop_movies = df1.sort_values('popularity', ascending=False)
pop_movies.head()

,title,tagline,overview,popularity,description
546,Minions,"Before Gru, they had a history of bad bosses","Minions Stuart, Kevin and Bob are recruited by...",875.581305,"Before Gru, they had a history of bad bosses M..."
95,Interstellar,Mankind was born on Earth. It was never meant ...,Interstellar chronicles the adventures of a gr...,724.247784,Mankind was born on Earth. It was never meant ...
788,Deadpool,Witness the beginning of a happy ending,Deadpool tells the origin story of former Spec...,514.569956,Witness the beginning of a happy ending Deadpo...
94,Guardians of the Galaxy,All heroes start somewhere.,"Light years from Earth, 26 years after being a...",481.098624,All heroes start somewhere. Light years from E...
127,Mad Max: Fury Road,What a Lovely Day.,An apocalyptic story set in the furthest reach...,434.278564,What a Lovely Day. An apocalyptic story set in...


In [ ]:
# localizando o ID, no nosso caso, do mais popular
movie_idx = np.where(movies_list == 'Minions')[0][0]
movie_idx

546

In [ ]:
# encontrando os filmes similares
movie_similarities = doc_sim_df.iloc[movie_idx].values
movie_similarities

array([0.01045346, 0.01072743, 0.        , ..., 0.00690641, 0.        ,
       0.        ])

In [ ]:
# selecionando os 5 mais populares IDs
similar_movie_idxs = np.argsort(-movie_similarities)[1:6]
similar_movie_idxs

array([506, 614, 241, 813, 154])

In [ ]:
# descrevendo os 5 filmes mais populares
similar_movies = movies_list[similar_movie_idxs]
similar_movies

array(['Despicable Me 2', 'Despicable Me',
       'Teenage Mutant Ninja Turtles: Out of the Shadows', 'Superman',
       'Rise of the Guardians'], dtype=object)

Ai tiraria até aqui
_________

# Construindo um recomendador de filmes

In [ ]:
# criando a função para o recomendador de filmes
def movie_recommender(movie_title,doc_sims, movies=movies_list): 
    # encontrando o ID do filme
    movie_idx = np.where(movies == movie_title)[0][0]
    
    # descrevendo os filmes mais similares
    movie_similarities = doc_sims.iloc[movie_idx].values
    
    # encontrando os IDs dos 5 primeiros filmes mais similares
    similar_movie_idxs = np.argsort(-movie_similarities)[1:6]
    # descrevendo os 5 filmes mais similares
    similar_movies = movies[similar_movie_idxs]

    # retornando com o resultado
    return similar_movies

In [ ]:
# criando lista dos 10 mais populares do nosso conjunto
popular_movies = pop_movies.title.tolist()[1:11]

In [ ]:
#Euclidiana
# recomendando os filmes para essa lista
for movie in popular_movies:
    print('Movie:', movie)
    print('Top 5 filmes recomendados:', movie_recommender(movie_title=movie,doc_sims=doc_eucl_df))
    print('\n**************************************************************************************')

Movie: Interstellar
Top 5 filmes recomendados: ['Alexander and the Terrible, Horrible, No Good, Very Bad Day'
 'Cold Mountain' 'Little Fockers' 'Dirty Work' 'The Lives of Others']

**************************************************************************************
Movie: Deadpool
Top 5 filmes recomendados: ['Mission: Impossible - Ghost Protocol' "Baby's Day Out"
 'The Replacements' 'Elf' 'Dude, Where’s My Car?']

**************************************************************************************
Movie: Guardians of the Galaxy
Top 5 filmes recomendados: ['Shanghai Knights' "Baby's Day Out" '21 Grams' 'Everest' 'Alex & Emma']

**************************************************************************************
Movie: Mad Max: Fury Road
Top 5 filmes recomendados: ['Anne of Green Gables' 'On Deadly Ground' 'Little Voice' 'The Gallows'
 'Pathology']

**************************************************************************************
Movie: Jurassic World
Top 5 filmes recomendados

In [ ]:
#Cosseno
# recomendando os filmes para essa lista
for movie in popular_movies:
    print('Movie:', movie)
    print('Top 5 filmes recomendados:', movie_recommender(movie_title=movie,doc_sims=doc_sim_df))
    print('\n**************************************************************************************')

Movie: Interstellar
Top 5 filmes recomendados: ['Gattaca' 'Space Pirate Captain Harlock' 'Space Cowboys'
 'Starship Troopers' 'Final Destination 2']

**************************************************************************************
Movie: Deadpool
Top 5 filmes recomendados: ['Silent Trigger' 'Underworld: Evolution' 'Bronson' 'Shaft' 'Don Jon']

**************************************************************************************
Movie: Guardians of the Galaxy
Top 5 filmes recomendados: ['Chasing Mavericks' 'E.T. the Extra-Terrestrial' 'American Sniper'
 'The Amazing Spider-Man 2' 'Hoop Dreams']

**************************************************************************************
Movie: Mad Max: Fury Road
Top 5 filmes recomendados: ['The 6th Day' 'Star Trek Beyond' 'Kites' 'The Orphanage'
 'The Water Diviner']

**************************************************************************************
Movie: Jurassic World
Top 5 filmes recomendados: ['Jurassic Park' 'The Lost World: 